# Sediment resuspension parameterization

Estimated as constant over time with the magnitudes as a function of tidal stresses.

In [1]:
import numpy as np
import datetime
import xarray as xr
import scipy.io
import sys
sys.path.append('../paper-materials/')
import mapping_functions as mf
from constants import imin, imax, jmin, jmax

%matplotlib inline

#### Load files:

In [3]:
# The Pb model configuration ANHA12 mask:
mesh         = xr.open_dataset('/ocean/brogalla/GEOTRACES/data/ANHA12/ANHA12_mask_Pb-20230213.nc')
mesh_lon     = mesh['nav_lon'].values
mesh_lat     = mesh['nav_lat'].values
tmask        = mesh['tmask'].values
mesh_bathy   = tmask[0,:,:,:]
bathy_masked = np.ma.masked_where((mesh_bathy > 0.1), mesh_bathy)
depths       = mesh['nav_lev']

In [4]:
# Tidal speed matrix provided by Jacqui-Lee Epstein and Stephanie Waterman (extracted from Carrere and Lyard, 2003)
mat         = scipy.io.loadmat('/ocean/brogalla/GEOTRACES/data/mean_tide_speed.mat')
tidal_speed = mat['mean_speed']
ts_mesh     = xr.open_dataset('/ocean/brogalla/GEOTRACES/data/hsquare.nc')
ts_lons     = ts_mesh['nav_lon'].values
ts_lats     = ts_mesh['nav_lat'].values

#### Calculations

In [5]:
tidal_speed_ANHA12 = mf.interp_np(ts_lons, ts_lats, tidal_speed, mesh_lon, mesh_lat)
tidal_speed_ANHA12[np.isnan(tidal_speed_ANHA12)] = 0
tidal_speed_ANHA12[tidal_speed_ANHA12 < 0.01]    = 0 # mask tidal speeds slower than 1 cm/s

In [ ]:
gamma     = 0.065
resus_cst = 1.0e-8

old_erosion_rate     = tidal_speed_ANHA12**2
log_old_erosion_rate = np.log10(old_erosion_rate)

erosion_rate     = gamma*(1-np.exp(-tidal_speed_ANHA12**2/gamma))
log_erosion_rate = np.log10(erosion_rate)

$ resuspension = 1 - e^{-v^2} \approx 1 - (1 - v^2) = v^2$

Write to file:

In [19]:
file_write = xr.Dataset(
    {'er_rate': (("y","x"), erosion_rate)}, 
    coords = {
        "y": np.zeros(2400),
        "x": np.zeros(1632),
    },
    attrs = {
        'long_name':'Erosion rate',
        'units':'kg/m2/s (when multipled by resus_cst)',
    }
)

file_write.to_netcdf('/ocean/brogalla/GEOTRACES/data/erosion_rate-Pb-20221102.nc')